In [12]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import warnings
warnings.filterwarnings('ignore')

In [17]:
df = pd.read_csv('credit_score_ds_1.csv')
df.head()


,month,age,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,num_of_loan,num_credit_inquiries,credit_history_age,amount_invested_monthly,...,occupation_Musician,occupation_Scientist,occupation_Teacher,occupation_Writer,payment_behaviour_High_spent_Large_value_payments,payment_behaviour_High_spent_Medium_value_payments,payment_behaviour_High_spent_Small_value_payments,payment_behaviour_Low_spent_Large_value_payments,payment_behaviour_Low_spent_Medium_value_payments,payment_behaviour_Low_spent_Small_value_payments
0,1,23.0,19114.12,1824.843333,3.0,4.0,4.0,4.0,265.0,21.46538,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2,23.0,19114.12,1824.843333,3.0,4.0,4.0,4.0,266.0,21.46538,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,23.0,19114.12,1824.843333,3.0,4.0,4.0,4.0,267.0,21.46538,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4,23.0,19114.12,1824.843333,3.0,4.0,4.0,4.0,268.0,21.46538,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,23.0,19114.12,1824.843333,3.0,4.0,4.0,4.0,269.0,21.46538,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [37]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

x = df.drop(['month', 'credit_score'], axis=1)
y = np.array(df['credit_score'])

scaler = MinMaxScaler()
scaler.fit(x)
t_x = scaler.transform(x)

X_train, X_test, y_train, y_test = train_test_split(t_x, y, test_size=0.2, random_state=42)


In [50]:

# Создание модели Sequential
model = Sequential()

# Добавление входного слоя с 10 нейронами (можно изменить на другое значение)
model.add(Dense(50, input_dim=31, activation='relu'))

# Добавление скрытого слоя с 8 нейронами и функцией активации ReLU
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))

# Добавление выходного слоя с 1 нейроном и функцией активации sigmoid (для бинарной классификации)
model.add(Dense(1, activation='sigmoid'))

# Компиляция модели с выбором оптимизатора (например, Adam) и функции потерь (binary_crossentropy для бинарной классификации)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Вывод структуры модели
model.summary()


Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_67 (Dense)            (None, 50)                1600      
                                                                 
 dense_68 (Dense)            (None, 100)               5100      
                                                                 
 dense_69 (Dense)            (None, 100)               10100     
                                                                 
 dense_70 (Dense)            (None, 50)                5050      
                                                                 
 dense_71 (Dense)            (None, 50)                2550      
                                                                 
 dense_72 (Dense)            (None, 1)                 51        
                                                                 
Total params: 24,451
Trainable params: 24,451
Non-tra

In [51]:
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Оценка модели на тестовом наборе
y_pred_proba = model.predict(X_test)

# Вычисление AUC
auc = roc_auc_score(y_test, y_pred_proba)

# Вывод результата
print(f'AUC: {auc}')

Epoch 1/50
2000/2000 [==============================] - 5s 2ms/step - loss: 0.4936 - accuracy: 0.7626 - val_loss: 0.4920 - val_accuracy: 0.7651
Epoch 2/50
2000/2000 [==============================] - 5s 2ms/step - loss: 0.4769 - accuracy: 0.7733 - val_loss: 0.4774 - val_accuracy: 0.7727
Epoch 3/50
2000/2000 [==============================] - 5s 2ms/step - loss: 0.4728 - accuracy: 0.7745 - val_loss: 0.4770 - val_accuracy: 0.7688
Epoch 4/50
2000/2000 [==============================] - 5s 2ms/step - loss: 0.4698 - accuracy: 0.7769 - val_loss: 0.4761 - val_accuracy: 0.7728
Epoch 5/50
2000/2000 [==============================] - 5s 2ms/step - loss: 0.4670 - accuracy: 0.7770 - val_loss: 0.4731 - val_accuracy: 0.7742
Epoch 6/50
2000/2000 [==============================] - 5s 2ms/step - loss: 0.4640 - accuracy: 0.7788 - val_loss: 0.4714 - val_accuracy: 0.7729
Epoch 7/50
2000/2000 [==============================] - 5s 2ms/step - loss: 0.4609 - accuracy: 0.7803 - val_loss: 0.4776 - val_accuracy:

In [58]:
best_auc = 0.0
best_hidden_size = None
best_num_epochs = None
# early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
h1 = [i for i in range(5.0, 100.0, 5.0)]
h2 = [i for i in range(5.0, 100.0, 5.0)]
h3 = [i for i in range(5.0, 100.0, 5.0)]
h4 = [i for i in range(5.0, 100.0, 5.0)]
h5 = [i for i in range(5.0, 100.0, 5.0)]
ep = [i for i in range(10, 100, 10)]

for h1 in h1:
    for h2 in h2:
        for h3 in h3:
            for h4 in h4:
                for h5 in h5:
                        for ep in ep:
                            model = Sequential()
                            model.add(Dense(h1, input_dim=31, activation='relu'))
                            model.add(Dense(h2, activation='relu'))
                            model.add(Dense(h3, activation='relu'))
                            model.add(Dense(h4, activation='relu'))
                            model.add(Dense(h5, activation='relu'))
                            model.add(Dense(1, activation='sigmoid'))
                            model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
                            history = model.fit(X_train, y_train, epochs=ep, batch_size=100, validation_split=0.2, callbacks=[early_stopping])
                            y_pred_proba = model.predict(X_test)
                            auc = roc_auc_score(y_test, y_pred_proba)
                            
                            if auc > best_auc:
                                                best_auc = auc
                                                best_hidden_sizes = (h1, h2, h3, h4, h5)
                                                best_num_epochs = ep

Epoch 1/10
640/640 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.7093 - val_loss: 0.5056 - val_accuracy: 0.7066
Epoch 2/10
640/640 [==============================] - 1s 2ms/step - loss: 0.4935 - accuracy: 0.7600 - val_loss: 0.4937 - val_accuracy: 0.7662
Epoch 3/10
640/640 [==============================] - 1s 2ms/step - loss: 0.4876 - accuracy: 0.7687 - val_loss: 0.4894 - val_accuracy: 0.7681
Epoch 4/10
640/640 [==============================] - 1s 2ms/step - loss: 0.4854 - accuracy: 0.7707 - val_loss: 0.4897 - val_accuracy: 0.7693
Epoch 5/10
640/640 [==============================] - 1s 2ms/step - loss: 0.4843 - accuracy: 0.7713 - val_loss: 0.4897 - val_accuracy: 0.7682
Epoch 6/10
640/640 [==============================] - 1s 2ms/step - loss: 0.4836 - accuracy: 0.7723 - val_loss: 0.4869 - val_accuracy: 0.7704
Epoch 7/10
640/640 [==============================] - 1s 2ms/step - loss: 0.4830 - accuracy: 0.7726 - val_loss: 0.4865 - val_accuracy: 0.7702
Epoch 

TypeError: 'int' object is not iterable

In [59]:
print(best_auc)
print(best_hidden_sizes)
print(best_num_epochs)

0.8019175733009655
(5, 5, 5, 5, 5)
60
